<a href="https://colab.research.google.com/github/jgxuann/Screening-Task/blob/main/Screening_Task_Guanxuan_Jiang.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ***Please proceed sequentially to ensure version consistency. Additionally, update your API and Google Drive addresses. If you wish to use my API, please keep it confidential and refrain from misuse. Thank you. Plase change your actual PATH, please contact me if you have any issuses during run my code.***

In [ ]:
# Please use this version
!pip install openai==0.28

In [ ]:
# Please use your google drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Dowload
!apt-get update
!apt-get install -y unrar tesseract-ocr
!apt-get install tesseract-ocr
!pip install openai pandas tqdm PyMuPDF pdfminer.six pytesseract pdf2image
!sudo apt-get update
!sudo apt-get install -y tesseract-ocr-chi-sim
!sudo apt-get install -y tesseract-ocr-eng
!sudo apt-get install -y poppler-utils
!pip install PyPDF2
!pip install openai
!pip install pandas

In [ ]:
import os
# Unzip
os.makedirs('/content/papers', exist_ok=True)

# Set .rar file path
rar_file_path = '/content/drive/MyDrive/32 article.rar'  # Change the value based on the actual path

# Unzip the RAR file to the 'papers' directory
!unrar x "{rar_file_path}" /content/papers/

# Lists all files in the 'papers/32 article' directory
for filename in os.listdir('/content/papers/32 article'):
    print(filename)


In [ ]:
import os
import PyPDF2
import openai
import pandas as pd
import json
from concurrent.futures import ThreadPoolExecutor, as_completed

# Set up the OpenAI API key, please use your own API key
# I am willing to let you use my test, but please do not abuse it
os.environ['OPENAI_API_KEY'] = 'sk-ahpAfeRW2xZh0Cg7144a0f8cC31d4e689d5aDb05E0BdC830'
os.environ['OPENAI_BASE_URL'] = "https://vip.yi-zhan.top/v1"

openai.api_key = os.getenv('OPENAI_API_KEY')
openai.api_base = os.getenv('OPENAI_BASE_URL')

In [ ]:
import re
# Path
pdf_folder_path = '/content/papers/32 article'

# Define the function to extract text from a PDF
def extract_text_from_pdf(pdf_path):
    try:
        with open(pdf_path, 'rb') as file:
            reader = PyPDF2.PdfReader(file)
            text = ""
            for page in reader.pages:
                page_text = page.extract_text()
                if page_text:
                    text += page_text
        return text
    except Exception as e:
        print(f"Error reading {pdf_path}: {e}")
        return ""

# Define the function to extract information using OpenAI
def extract_information(text):
    prompt = f"""
    Please extract the following ten key points from the given research article text and return them in a structured JSON format:

    1. Context: Specify whether the study is focused on a specific industry, task or a broader, conceptual scope.
    2. Research Question and Findings: Identify the main research question and summarize the key findings.
    3. Theme of Research:
        a. Human vs. AI: Highlight the comparative advantages between humans and AI, including any conditional outcomes or superiority in specific contexts.
        b. Human + AI Collaboration: Specify the types of collaboration discussed, such as the roles of humans and AI, and the sequence of their actions in the process.
    4. Method: Classify the research method into one of the following three categories:
        a. Conceptual/Case Study
        b. Modeling (including stylized or Operations Research (OR) models)
        c. Empirical Research (laboratory/field experiments or secondary data analysis)
    5. Contribution: Determine the main contribution of the research, categorizing it as theoretical, managerial, or methodological.
    6. Future Potential and Limitations: Summarize the recommendations for future research directions or the limitations of the study.
    7. Keywords: Conclude the key words.
    8. Authors: List all authors.
    9. Publication Information: Find the Jornal/Conference infomation(pages,name and so on) and years.
    10. Article Name: List the name of the article

    Here is the text of the research article:
    {text}

    Please provide the extracted information in pure JSON format without any code blocks, comments, or additional text:

    {{
        "Context": "",
        "Research Question and Findings": "",
        "Theme of Research": {{
            "Human vs AI": "",
            "Human + AI Collaboration": ""
        }},
        "Method": "",
        "Contribution": "",
        "Future Potential and Limitations": "",
        "Keywords":"",
        "Authors":"",
        "Publication Information":"",
        "Article Name":""
    }}
    """

    try:
        response = openai.ChatCompletion.create(
            model="gpt-4o",  # Correct model name
            messages=[
                {"role": "system", "content": "You are a helpful assistant that extracts structured information from research articles."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.3,
            max_tokens=10000,
        )
        result = response['choices'][0]['message']['content']

        # Debug: Print the raw output of the GPT
        print("GPT Original Output:", result)

        # Extract JSON parts using regular expressions
        json_pattern = re.compile(r'\{.*\}', re.DOTALL)
        match = json_pattern.search(result)
        if match:
            json_str = match.group()
            try:
                data = json.loads(json_str)
                return data
            except json.JSONDecodeError as e:
                print("JSON decoding error. Please check the extracted JSON string.")
                print("Extracted JSON string:", json_str)
                return None
        else:
            print("No JSON content found in GPT's response.")
            return None

    except Exception as e:
        print(f"Error calling OpenAI API: {e}")
        return None

def process_pdfs(pdf_folder_path, max_pdfs=2):
    """
    Process PDF files in a specified directory to extract summaries.

    Args:
        pdf_folder_path (str): Path to the directory containing PDF files.
        max_pdfs (int): Maximum number of PDF files to process.

    Returns:
        list: List of dictionaries containing filenames and their corresponding summaries.
    """
    all_summaries = []
    count = 0

    # Iterate over all files in the specified directory
    for filename in os.listdir(pdf_folder_path):
        if filename.lower().endswith('.pdf'):
            pdf_file_path = os.path.join(pdf_folder_path, filename)
            print(f"Processing {pdf_file_path} ...")

            # Extract text from the PDF
            text = extract_text_from_pdf(pdf_file_path)
            if not text:
                print(f"Unable to process {pdf_file_path} due to extraction issues.\n")
                continue

            # Extract structured information using OpenAI
            info = extract_information(text)
            if info:
                # Add the extracted information to the summaries list
                all_summaries.append({
                    "filename": filename,
                    "summary": info
                })

                # Print the extracted information
                print("Extracted Information:")
                print(json.dumps(info, ensure_ascii=False, indent=4))
                print("Summary added to the summaries list.\n")
            else:
                print("Failed to extract valid information.\n")

            count += 1
            if count >= max_pdfs:
                print(f"Reached the maximum number of PDFs to process: {max_pdfs}. Stopping processing.")
                break

    return all_summaries

def save_summaries_to_json(summaries, output_directory='/content', output_filename="summaries.json"):
    """
    Save all summaries to a JSON file.

    Args:
        summaries (list): List of dictionaries containing summaries.
        output_directory (str): Directory where the JSON file will be saved.
        output_filename (str): Name of the output JSON file.
    """
    output_file_path = os.path.join(output_directory, output_filename)
    try:
        with open(output_file_path, 'w', encoding='utf-8') as json_file:
            json.dump(summaries, json_file, ensure_ascii=False, indent=4)
        print(f"All summaries have been saved to {output_file_path}")
    except Exception as e:
        print(f"Error saving summaries to file: {e}")

def main():
    """
    Main function to execute the PDF processing and summary extraction.
    """
    # Define the path to the folder containing PDF files
    pdf_folder_path = '/content/papers/32 article'  # Replace with your actual PDF folder path

    # Define the maximum number of PDFs to process
    max_pdfs = 32  # Set the number of PDFs you want to process

    # Process the PDFs and extract summaries
    all_summaries = process_pdfs(pdf_folder_path, max_pdfs=max_pdfs)

    # Save the summaries to a JSON file
    save_summaries_to_json(all_summaries, output_directory='/content', output_filename="summaries.json")

if __name__ == "__main__":
    main()

Processing /content/papers/32 article/retrieve-3.pdf ...
GPT Original Output: {
    "Context": "The study is focused on the call center customer service industry.",
    "Research Question and Findings": "The main research question is how the introduction of voice-based AI systems impacts call length, customers' demand for human service, and customer complaints in call center customer services. The key findings are that the AI system temporarily increases call length and demand for human service but persistently reduces customer complaints. The AI system is more effective for simple requests and for older, female, and experienced customers.",
    "Theme of Research": {
        "Human vs AI": "The AI system reduces customer complaints more effectively than the traditional IVR system, especially for simple service requests. However, speech-recognition failures can lead to increased demand for human service and complaints.",
        "Human + AI Collaboration": "The study discusses the role

# ***The demo and summary for Function1 are executed here. Please ensure that all prior tasks are completed in sequence up to this point to facilitate a smooth process.***


In [ ]:
import json
import re

# Define the path to summaries.json
summaries_file_path = '/content/summaries.json'

# Load summaries.json
# Load summaries.json
try:
    with open(summaries_file_path, 'r', encoding='utf-8') as f:
        summaries = json.load(f)
except FileNotFoundError:
    print(f"File not found: {summaries_file_path}")
    summaries = []

# Define a function to find summaries by a specific author
def find_summary_by_author(author_keyword):
    matched_summaries = []
    for summary in summaries:
        authors = summary['summary'].get('Authors', '')
        # Use regular expressions for more accurate author matching
        if re.search(rf'\b{re.escape(author_keyword)}\b', authors, re.IGNORECASE):
            matched_summaries.append(summary)
    return matched_summaries

# Find all summaries containing 'Gnewuch'
gnewuch_summaries = find_summary_by_author('Gnewuch')

if gnewuch_summaries:
    print(f"### Found {len(gnewuch_summaries)} summaries by 'Gnewuch':\n")
    # Collect all summaries' content to integrate
    combined_summary_content = ""
    for idx, gnewuch_summary in enumerate(gnewuch_summaries, start=1):
        print(f"### Gnewuch et al. Summary {idx}:")
        print(json.dumps(gnewuch_summary['summary'], ensure_ascii=False, indent=4))
        print("\n---\n")
        # Combine the summary content for AI processing
        combined_summary_content += json.dumps(gnewuch_summary['summary'], ensure_ascii=False, indent=4) + "\n"

    # Define a function to generate an integrated summary using OpenAI's GPT
    def generate_integrated_summary(content):
        try:
            response = openai.ChatCompletion.create(
                model="gpt-4o",
                messages=[
                    {"role": "system", "content": "You are a helpful assistant that summarizes academic research."},
                    {"role": "user", "content": f"Please create a comprehensive summary based on the following content:\n\n{content}"}
                ],
                max_tokens = 10000,
                temperature = 0.3,
            )
            return response.choices[0].message['content'].strip()
        except Exception as e:
            print(f"An error occurred while generating the summary: {e}")
            return None

    # Generate the integrated summary
    integrated_summary = generate_integrated_summary(combined_summary_content)

    if integrated_summary:
        print("### Integrated Summary Generated by AI:\n")
        print(integrated_summary)
    else:
        print("Failed to generate the integrated summary.")
else:
    print("No summaries containing 'Gnewuch' were found. Please check the file naming and Authors field format.")

# Initialize counters
conceptual_count = 0
modeling_count = 0
empirical_count = 0

experiments_count = 0
secondary_data_count = 0

# Iterate through all summaries and count
for summary in summaries:
    method = summary['summary'].get('Method', '').lower()

    if 'conceptual' in method or 'case study' in method:
        conceptual_count += 1
    if 'modeling' in method:
        modeling_count += 1
    if 'empirical' in method:
        empirical_count += 1
        # Further categorize empirical research
        if 'experiment' in method:
            experiments_count += 1
        if 'secondary data' in method or 'secondary data analysis' in method:
            secondary_data_count += 1

# Print the counting results
print("\n### Research Methods Count in 32 Papers:")
print(f"- Conceptual Studies: {conceptual_count} papers")
print(f"- Modeling Studies: {modeling_count} papers")
print(f"- Empirical Studies: {empirical_count} papers")

print("\n### Within Empirical Studies:")
print(f"- Experiments: {experiments_count} papers")
print(f"- Secondary Data Analysis: {secondary_data_count} papers")

# **Additional Summary Statement**
print("\n### Summary Statement:")
print(f"Among the 32 papers, {conceptual_count} are conceptual studies, {modeling_count} are modeling studies (either stylized or OR models), and {empirical_count} are empirical studies. Of the empirical studies, {experiments_count} are experiments, and {secondary_data_count} use secondary data analysis.")

## ***Function 2 Demo***

In [ ]:
import json
import openai

# Define the path to summaries.json
summaries_file_path = '/content/summaries.json'

# Load summaries.json
def load_summaries(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            return json.load(f)
    except FileNotFoundError:
        print(f"File not found: {file_path}")
        return []

# Filter function
def simple_filter_summaries(summaries, theme_keyword, method_keyword):
    """
    Filters summaries to include only those that contain the specified theme keyword and method keyword.

    Parameters:
        summaries (list): List of all literature summaries.
        theme_keyword (str): Theme keyword, e.g., 'Human vs AI'.
        method_keyword (str): Method keyword, e.g., 'Empirical Research'.

    Returns:
        list: Filtered list of literature summaries.
    """
    filtered = []
    for summary in summaries:
        theme_of_research = summary['summary'].get('Theme of Research', {})
        method = summary['summary'].get('Method', '').lower()

        # Check if the specified theme keyword exists in the theme keys
        has_theme = theme_keyword.lower() in (key.lower() for key in theme_of_research.keys())

        # Check if the method field contains the specified keyword
        has_method = method_keyword.lower() in method

        if has_theme and has_method:
            filtered.append(summary)
    return filtered

# Generate literature review function
def generate_literature_review(filtered_summaries):
    """
    Generates a literature review and reference list based on the filtered summaries.

    Parameters:
        filtered_summaries (list): List of filtered literature summaries.

    Returns:
        None
    """
    if not filtered_summaries:
        print("No articles meet the criteria.")
        return

    compiled_content = ""
    reference_list = []
    for summary in filtered_summaries:
        compiled_content += json.dumps(summary['summary'], ensure_ascii=False, indent=4) + "\n"
        title = summary['summary'].get('Article Name', 'Title Missing')
        authors = summary['summary'].get('Authors', 'Authors Missing')
        publication_info = summary['summary'].get('Publication Information', 'Journal/Conference Missing')
        if ',' in publication_info:
            parts = publication_info.split(',')
            journal = parts[0].strip()
            year = parts[1].strip() if len(parts) > 1 else 'Year Missing'
        else:
            journal = publication_info.strip()
            year = 'Year Missing'
        reference = f"{authors}. ({year}). {title}. {journal}."
        reference_list.append(reference)

    # Adjusted prompt to emphasize narrative style and natural citation integration
    prompt = (
        "You are an assistant proficient in writing academic literature reviews. "
        "Based on the following content, please write a comprehensive and cohesive literature review that seamlessly integrates the provided studies. "
        "Discuss the connections, common themes, trends, and future directions of the research in a narrative format without using bullet points or separate sections. "
        "Ensure that all references are cited appropriately within the text using APA in-text citations.\n\n"
        f"{compiled_content}"
    )

    try:
        response = openai.ChatCompletion.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": "You are a knowledgeable assistant skilled in writing academic literature reviews."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=15000,
            temperature=0.5,
        )
        literature_review = response.choices[0].message['content'].strip()
    except Exception as e:
        print(f"Error generating literature review: {e}")
        return

    if literature_review:
        # Format the reference list (APA style)
        formatted_references = "\n".join([f"{idx + 1}. {ref}" for idx, ref in enumerate(reference_list)])
        full_review = f"{literature_review}\n\n### References:\n{formatted_references}"
        print(full_review)
    else:
        print("Failed to generate literature review.")

# Main function
def main():
    summaries = load_summaries(summaries_file_path)
    theme_keyword = 'Human vs AI'
    method_keyword = 'Empirical Research'
    filtered = simple_filter_summaries(summaries, theme_keyword, method_keyword)

    if not filtered:
        print("No articles meet the criteria.")
        return

    generate_literature_review(filtered)

if __name__ == "__main__":
    main()
